<a href="https://colab.research.google.com/github/DarshanPatel0919/Deep-Learning/blob/master/LabAssignment6_201701436.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [ ]:
import numpy as np
from matplotlib.pyplot import *

#Function, Gradient and Global Minima on Contour Plot

In [ ]:
def func(x,y):
  return (x-y+1.5)**2 + ((2.5+x) - x*y*y)**2 + ((2.625 - x) + x*y*y*y)**2

def custom_function(t):
  x,y = t
  return func(x,y)

def gradient(t):
  x,y = t
  dfdx = 2*((x-y+1.5) + (2.5+x - x*y*y)*(1-y*y) + (2.625 - x + x*y*y*y)*(-1 + y*y*y))
  dfdy = 2*((x-y+1.5)*(-1) + (2.5+x - x*y*y)*(-2*x*y) + (2.625 - x + x*y*y*y)*(3*x*y*y))
  return np.array([dfdx,dfdy])

In [ ]:
#Global Minima and Contour Plot

# Below function is used to make sure all the levels of contours look uniform in the plot
def normalized_levels(Z, k):
    n, m = Z.shape
    size = n*m
    step = size//k
    s = np.sort(Z,1)
    level = []
    for i in range(0,size,step):
      level += [s[i//m,i%m] , s[i//m,-(i%m)-1]]
    level = list(dict.fromkeys(sorted(level)))
    return level

st, en = -2,3
r = 500
n = int((en-st)*r) + 1
x, y = np.linspace(st,en,n), np.linspace(st,en,n)

X, Y = np.meshgrid(x, y)
Z = func(X, Y)

mn = np.min(Z)
idy, idx = np.where(Z==mn)
minx, miny = np.around(x[idx],5), np.around(y[idy],5)
p = list(zip(minx, miny))
p = list(dict.fromkeys(p))[0]

print("Optimum Solution: ",p)
print("Minimum Value: ",mn)

## Plots the function's contour diagram
def plot_function(name = 'Function', xset=[-2,3], yset=[-2,3], r=500, xr=[-2,3], yr=[-2,3]):
  ax = gca()
  #ax.set_aspect('equal')
  
  n = int((en-st)*r) + 1
  x, y = np.linspace(xset[0],xset[1],1+int((xset[1]-xset[0])*r)), np.linspace(yset[0],yset[1],int((yset[1]-yset[0])*r))

  X, Y = np.meshgrid(x, y)
  Z = func(X, Y)
  
  ax.contour(X, Y, Z, normalized_levels(Z, 8),colors=['skyblue'])
  grid()
  xlabel('x')
  ylabel('y')
  xlim(xr)
  ylim(yr)
  title(name)

## Plot the path followed by given optimizer

## Note. Star: Indicates Final point of path
## Note. O :  Indicates Initial point of Path
def plot_path(path, color = 'r', name='', st=50, en=100):
  plot(path[:,0],path[:,1],color,label=name)
  scatter(path[0,0], path[0,1], marker='o', s=st, c=color)
  scatter(path[-1,0], path[-1,1], marker='*', s=en, c=color)

figure(figsize=(5,5))
plot_function()
plot_path(np.array([p]))
show()

In [ ]:
# Critical Points
print(func(0.502803,-1.56081))
print(func(-0.151051,2.12316))
print(func(0.29767,-0.83626))

#Optimizers

In [ ]:
## Difference between previous and cuurent state of gradient decent
def error(x,y):
  return np.sqrt(np.mean((x-y)**2))

def print_ans(p, f , n):
  print("Point: ",np.around(p,4))
  print("Value: ",np.around(f(p),4))
  print("Iterations: ",n, '\n')

def SGD(w1, alpha = 0.01, n= 10000, eps = 1e-8):

  ## Store each point found during SGD in 'path'
  path = []

  ## Maximum number of iterations (i.e. EPOCHS) is 'n'
  while len(path)<n:

      path += [w1]
      gd = gradient(w1)
      w2 = w1 - alpha*gd

      ## If the difference is less than given threshold stop the algorithm
      if error(w1,w2) < eps:
        break

      ## w2 will be the previous weight for the next iteration
      w1 = w2

  ## Return Optimum Solution along with the path
  return (w1,path)

def Momentum_NAG(w1, alpha= 0.01, gamma = 0.9, n= 10000, eps = 1e-8):
  path = []
  d0 = 0
  while len(path)<n:
    path += [w1]
    w_adv = w1 - alpha*d0
    gd = gradient(w_adv)
    d1 = gamma*d0  + (1-gamma)*gd
    w2 = w1 - alpha*d1
    if error(w1,w2) < eps:break
    w1,d0 = w2,d1
  return (w1,path)

def RMSProp(w1, alpha=0.01, beta= 0.9, n= 100000, eps= 1e-8):
  path = []
  h0 = 0
  while len(path)<n:
      path += [w1]
      gd = gradient(w1)
      h1 = beta*h0 + (1-beta)*(gd**2)
      w2 = w1 - alpha*gd/(np.sqrt(h1)+eps)
      if error(w1,w2) < eps:break
      w1,h0 = w2,h1
  return (w1,path)

def adam(w1, alpha=0.1, beta1= 0.9, beta2= 0.99, n= 10000, eps= 1e-8):
  path = []
  d0,h0 = 0,0
  while len(path)<n:
      path += [w1]
      gd = gradient(w1)
      d1 = beta1*d0 + (1-beta1)*gd
      h1 = beta2*h0 + (1-beta2)*(gd**2)
      w2 = w1 - alpha*d1/(np.sqrt(h1)+eps)
      if error(w1,w2) < eps:break
      w1,d0,h0 = w2,d1,h1
  return (w1,path)
  
tester = custom_function

## Initial Weights
w0 = np.array([-2,0])

name = ['SGD','Momentum(NAG)','RMSprop','ADAM']
optimizer = [SGD, Momentum_NAG, RMSProp, adam]

figure(figsize=(20,4.5))
PATH = {}
for i in range(len(name)):
  print("========  " + name[i] + "  =========")
  subplot(1,4,i+1)
  w, path = optimizer[i](np.copy(w0))
  PATH[i] = np.copy(path)
  print_ans(w, tester, len(path))
  plot_function(name[i])
  plot_path(np.array(path))
show()

figure(figsize=(20,5))
for i in range(len(name)):
  subplot(1,4,i+1)
  plot_function(name[i],xset=[-0.6,0],yset=[1.5,2.2],xr=[-0.6,0],yr=[1.5,2.2])
  plot_path(np.array(PATH[i]),st=5,en=5)
suptitle('Closer Look')
show()

**Observation**

* By Looking at the Contour Plots we observe that the function near minima is very very flat. 
* SGD shows the smoothest (also slowest) treanstion towards the minima
* Momentum Accelarated Gradient decent is Faster compared to SGD always it is relatively smoother and stable than next two optimizers.
* Root Mean Square Propogation based optimizer is actually converging faster but as it reaches near the minima frequency of zigzag is so much that it never stays on the minima and keeps zig-zagging around it, hence it keeps on going even if we set iterations to 1 Million it does not converge on one point, but it is assured that it will zig zag very near to minima, so we can apply **normal gradient decent** after fewer iterations of RMS Propagation and get the best out of it.
* Adam is also shows zig zag behaviour near the minima and along the path, but it is controlled by both velocity and momentum hence it will converge near the global minima, or in other words we are combining the advatages of Root Mean Square Propagation method and Momentum Accelarated method. Also it overcomes the drawback of SGD since it very faster compared to all of the previous optimizers
* Intuitively we might think that learning rate could be increased for other three optimizer and get the better results, but it does not work as well since it results in divergence. It is because ADAM is considering both the cases it does not diverge, hence ADAM is the best optimizer as compared to previous functions.
* Note that for very sensitive functions ADAM might show diverging behaviour in that case we need to choose appropriate value of learning rate, but for most of the time ADAM works best for alpha = 0.1

#Comparison

In [ ]:
colors = ['r','g','b','y']
figure(figsize=(5,5))
plot_function('Compare',xr=[-2,0.5],yr=[-1,2.5])
for i in range(len(name)):
  path = PATH[i]
  plot_path(np.array(path),colors[i],name[i])
legend(bbox_to_anchor=(1.5, 1))
show()

* Here we can see that how steady stochastic gradient decent is, but it is very much slower.
* Where as ADAM shows unsteady behaviour at first, but it coverges to minima very faster, very helpful for solving problems like vanishing gradient.
* Momentum Accelaration Based approach led us to RMS Propagation and ADAM. 
* Momentum and RMS prop performances are between ADAM and SGD. Using RMS Prop along with SGD might be the trick to improve its performance.